In [1]:
# Data Analysis Libraries
import pandas as pd
import numpy as np
import math
import string
import time

# Web Scrapping Libraries
import requests
from bs4 import BeautifulSoup

#pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns',None)

In [2]:
def dados_iniciais_livro(i):
    
    url_resenha = 'https://www.skoob.com.br/livro/resenhas/{}/mais-gostaram/mpage:1'.format(i) 
    # página do Skoob em que constam as resenhas, em que o primeiro {} será o número do livro e o segundo {} a página de reviews
    
    response = requests.get(url_resenha.format(i))
    pagina_html = BeautifulSoup(response.text)
    corpo = pagina_html.findAll(id="corpo")[0] # corpo da página a ser feita a raspagem de dados (Web Scraping)    
    
    Livro = corpo.findAll('strong')[0].get_text() # Nome do Livro
    try:
        Autor = corpo.findAll('a')[1].get_text() # Nome do Autor     
    except:
        Autor = None
    
    if Autor == '\nR$ \n':
        Autor = corpo.findAll('i')[0].get_text()
    try:
        ISBN_13 = corpo.findAll('div', {"class": "sidebar-desc"})[0].find('span').get_text()
    except:
        ISBN_13 = None
    try:
        lancamento = int(corpo.findAll('div', {"class": "sidebar-desc"})[0].get_text().split('Ano: ')[-1].split(' / Páginas:',1)[0])
    except:
        lancamento = None
    Paginas = corpo.findAll('div', {"class": "sidebar-desc"})[0].get_text().split('Páginas: ')[-1].split(' ',1)[0]    
    try:
        total_resenhas = int(corpo.findAll(id="perfil-conteudo-intern")[0].findAll('b')[0].get_text().split(' ')[0])
    except:
        total_resenhas = 0
    
    print('Livro {}: {}'.format(i, Livro))
        
    return(Livro, Autor, ISBN_13, lancamento, Paginas, total_resenhas, url_resenha)

In [3]:
def detalhes_do_livro(i, Livro):
    
    titulo_tratado = Livro
    for c in string.punctuation:
        titulo_tratado = titulo_tratado.replace(c,'')
    
    url_livro = ('https://www.skoob.com.br/'+titulo_tratado+'-'+str(i)).replace(' ','-').lower().replace('--','-')
    
    response2 = requests.get(url_livro)
    pagina_html2 = BeautifulSoup(response2.text)
    corpo2 = pagina_html2.findAll(id="corpo")[0] # corpo da página a ser feita a raspagem de dados (Web Scraping)
    detalhes = corpo2.findAll(id='pg-livro-principal-container')[0]
    
    Rating = float(detalhes.find(id="pg-livro-box-rating").find('span').get_text())
    Avaliacoes = int(detalhes.find(id="pg-livro-box-rating-avaliadores-numero").get_text().split(' ')[0].replace('.',''))
    
    total_resenhas_real = int(detalhes.findAll('div',{'class':'bar'})[0].findAll('a')[1].get_text().replace('.',''))
    abandonos = int(detalhes.findAll('div',{'class':'bar'})[1].findAll('a')[1].get_text().replace('.',''))
    relendo = int(detalhes.findAll('div',{'class':'bar'})[2].findAll('a')[1].get_text().replace('.',''))
    querem_ler = int(detalhes.findAll('div',{'class':'bar'})[3].findAll('a')[1].get_text().replace('.',''))
    lendo = int(detalhes.findAll('div',{'class':'bar'})[4].findAll('a')[1].get_text().replace('.',''))
    leram = int(detalhes.findAll('div',{'class':'bar'})[5].findAll('a')[1].get_text().replace('.',''))
    
    
    try:
        generos = detalhes.find(id='livro-perfil-sinopse-txt').find('span').get_text()
    except:
        generos = None
    if generos != None:
        descricao = detalhes.find(id='livro-perfil-sinopse-txt').find('p').get_text().split(generos)[0]
        generos = generos.strip().split(' / ')
    else:
        descricao = detalhes.find(id='livro-perfil-sinopse-txt').find('p').get_text()
    
    
    return(descricao, generos, Rating, Avaliacoes, total_resenhas_real, abandonos, relendo, querem_ler, lendo, leram, url_livro)

In [4]:
def data_de_lancamento(i, lancamento):
    url_edicoes = 'https://www.skoob.com.br/livro/edicoes/'+str(i)
    
    response3 = requests.get(url_edicoes)
    pagina_html3 = BeautifulSoup(response3.text)
    corpo3 = pagina_html3.findAll(id="corpo")[0].findAll('div',{'style':'margin-top:10px;'})[0] # corpo da página a ser feita a raspagem de dados (Web Scraping)
    
    numero_edicoes = len(corpo3.findAll('div',{'style':'float:left; width:180px;'}))
    lancamentos = []
    for edicoes in range(0,numero_edicoes):
        lancamentos.append(int(corpo3.findAll('div',{'style':'float:left; width:180px;'})[edicoes].get_text().split('Ano: ')[-1].split('Páginas:')[0]))
    try:
        if min(lancamentos) < lancamento:
            lancamento = min(lancamentos)
    except: 
        pass
    return(lancamento)

In [5]:
def obter_resenhas(i, Livro, total_resenhas, limite_resenhas, imprimir):
    
    lista_reviews = [] # lista com output de todas as reviews coletadas
    
    url_resenha = 'https://www.skoob.com.br/livro/resenhas/{}/mais-gostaram/mpage:{}'
    
    total_pags = math.ceil(total_resenhas/15)
    
    if limite_resenhas == 0: # se limite for definido como 0, pegará todas os comentários existentes
        pass   
    elif total_pags > math.ceil(limite_resenhas/15): # se houverem mais páginas que as necessárias para encontrar o limite
        total_pags = math.ceil(limite_resenhas/15) # limitará as páginas de modo a limitar o total de comentários
    else:
        pass # se limite_resenhas não for 0 nem maior que o existente, teremos menos reviews que o desejado, portanto pegaremos todos os disponíveis
    
    contador_reviews  = 0 # definindo um contador de comentários por livro
        
    for paginas_de_resenha in range(1,total_pags+1): # range das páginas com comentários, seguindo limite caso tenha sido definido
        response4 = requests.get(url_resenha.format(i,paginas_de_resenha))
        pagina_html4 = BeautifulSoup(response4.text)
        corpo4 = pagina_html4.findAll(id="corpo")[0] # corpo das reviews de cada página  
        
        
        if imprimir: # se True, irá printar o livro e página que está sendo coletado
            print('Livro {}: {} - Página: {}'.format(i, Livro, paginas_de_resenha)) # printando página da review, para identificar casos de bug

        # existem até 15 reviews por página, cada uma com o ID da review aparecendo 2x, portanto
        # o range padrão de coleta de reviews por página vai de 0 a 29, pulando de 2 em 2
        # no entanto, possívelmente ele foi limitado    
    
        if (paginas_de_resenha == math.ceil(limite_resenhas/15) and limite_resenhas!= 0): # se a página atual for a mesma definida ao limitar o numero de reviews
            ultima_review = (limite_resenhas - contador_reviews)*2 # encontrar quantas reviews ainda serão coletadas para atingir limite definido
            # multiplicar por 2 uma vez que pularemos de 2 em 2    
        else:
            ultima_review = 30 # se nao estiver no limite definido, usará o range completo das reviews 
            
        for resenhas in range(0,ultima_review,2):
            try: # testar se existe reviews nesta posição, uma vez que se o total não tiver sido limitado,
                # na última página de reviews pode haver menos de 15 reviews
                if imprimir: # se True, irá printar o livro e página e review que está sendo coletado
                    print('Livro {}: {} - Página: {} - Review: {}'.format(i, Livro, paginas_de_resenha,int(resenhas/2+1))) # printando a posição da review, para identificar casos de bug
                          
                # dados gerais da resenha    
                review_pt1 = corpo4.findAll(id="perfil-conteudo-intern")[0].findAll("div", id=lambda value: value and value.startswith("resenha"))[resenhas]
                id_resenha = review_pt1.get('id') # ID da review
                id_usuario = review_pt1.find('a').get('href').split('/')[-1] # ID do usuário
                nota = float(review_pt1.find('star-rating').get('rate')) # Nota dada pelo usuário
                           
                # dados de texto da resenha
                review_pt2 = corpo4.findAll(id="perfil-conteudo-intern")[0].findAll("div", id=lambda value: value and value.startswith("resenha"))[resenhas+1]
                            
                try: # caso exista um título, salvar ele separadamente
                    titulo = review_pt2.findAll('strong')[1].get_text() 
                except IndexError: # caso não exista um título, salvar None
                    titulo = None    
                    
                resenha = review_pt2.get_text() # obtendo o texto que consta título, data e resenha
                # na estrutura obtida, caso exista um titulo, ele é a ultima informação antes da resenha em si
                # caso não exista um título, a data é a ultima informação                
    
                if titulo == None: # caso não haja um título, usar a data como divisor para obter apenas a resenha
                    resenha = resenha.split('/',2)[-1][4:]
                else: # caso haja um título, utilizá-lo como divisor para obter a resenha
                    resenha = resenha.split(titulo,1)[-1]    
                livro_pagina_posicao = '{}-{}-{}'.format(i,paginas_de_resenha,(resenhas/2+1))
                
                lista_reviews.append({'review_id': id_resenha,
                                      'user_id': id_usuario,
                                      'livro_pagina_posicao':livro_pagina_posicao,
                                      'rating': nota,
                                      'review_title': titulo,
                                      'review': resenha})
            except:
                break
                
    return(lista_reviews)

In [6]:
def raspar_skoob(livro_inicial, livro_final, limite_resenhas,imprimir):
    # livro_inicial: primeiro livro a ser coletado (número inteiro não nulo)
    # livro_final: ultimo livro a ser coletado, incluindo-o (sendo necessáriamente um número maior que o do livro_inicial)
    # limite_resenhas: limite de reviews a serem coletados. 
        # Caso o valor seja menor que o número de reviews existentes, ele limitárá o total
        # Caso insira um valor maior que o número de reviews disponíveis, ele pegará o máximo existente
        # Caso não deseje limitar, inserir o valor 0
    # imprimir: se True, irá printar os estágios, se False, não irá printar os estágios
    
    
    contagem = 0 # contagem para definir saída do loop caso não existam 15 livros em sequência, um indício de que chegou ao final dos livros existentes
    
    lista_livros = [] # lista dos livros coletados    
    
    
    # range da lista de livros, começando no livro_inicial até livro_final, em que será feita a coleta
    for i in range(livro_inicial,livro_final+1):     
    
        if contagem == 15:
            print('\nHá {} tentativas não foi encontrado um livro, portanto, scraping será interrompido!'.format(contagem))
            break

        while True: # enquanto for true, seguira tentando obter os dados daquele único livro i
            try: # testar se existe um livro naquela posição, caso não exista simplesmente passará para o próximo livro
                lista_reviews=[]
                contagem = 0 # se o try funcionar, foi encontrado um livro, portanto, zerar contador
                
                # chamar a função dados_iniciais_livro
                Livro, Autor, ISBN_13, lancamento, Paginas, total_resenhas, url_resenha = dados_iniciais_livro(i)
                
                # chamar a função detalhes_do_livro
                descricao, generos, Rating, Avaliacoes, total_resenhas_real, abandonos, relendo, querem_ler, lendo, leram, url_livro = detalhes_do_livro(i, Livro) 
                
                # chamar a função da data_de_lancamento
                lancamento = data_de_lancamento(i, lancamento)                                             
                
                if total_resenhas != 0: # caso existam resenhas para este livro
                    lista_reviews = obter_resenhas(i, Livro, total_resenhas, limite_resenhas, imprimir)                      
                    
                else: # caso não exista nenhuma resenha 
                    lista_reviews.append({'review_id': None,
                                          'user_id': None,
                                          'livro_pagina_posicao':'{}-{}-{}'.format(i,1,0),
                                          'rating': None,
                                          'review_title': None,
                                          'review': None
                                         })
                # Salvar o livro após coleta
                lista_livros.append({'autor':Autor,
                                     'titulo':Livro,
                                     'numero_do_livro':i,
                                     'generos':generos,
                                     'data_lancamento':lancamento,
                                     'ISBN_13':ISBN_13,
                                     'paginas':Paginas,
                                     'nota_media':Rating,
                                     'total_de_avaliacoes':Avaliacoes,
                                     'leram':leram,
                                     'lendo':lendo,
                                     'querem_ler':querem_ler,
                                     'relendo':relendo,
                                     'abandonos':abandonos,
                                     'total_resenhas':total_resenhas_real,
                                     'url_livro':url_livro,
                                     'url_resenha':url_resenha,
                                     'descricao':descricao,
                                     'reviews':lista_reviews
                                    })
                break
                    
                    
            except IndexError: # caso não exista um livro nesta página, passar para próximo livro
                contagem +=1 # adicionar 1 ao contador para cada livro não existente
                break
                
            except requests.exceptions.ConnectionError: # caso dê erro de internet, continuar tentando até a internet voltar
                continue        
            except requests.exceptions.ChunkedEncodingError: # caso dê erro de internet, continuar tentando até a internet voltar
                continue           
       #     except:
       #         print('Um erro ocorreu no livro {}'.format(i))
       #         contagem +=1
       #         break
    print('\n')
    print (time.asctime(time.localtime(time.time())))    
    return(lista_livros)

In [7]:
print (time.asctime(time.localtime(time.time())))

Sat Mar 12 19:52:18 2022


In [8]:
livros = raspar_skoob(1,10,10,True)

Livro 1: Ensaio Sobre a Cegueira
Livro 1: Ensaio Sobre a Cegueira - Página: 1
Livro 1: Ensaio Sobre a Cegueira - Página: 1 - Review: 1
Livro 1: Ensaio Sobre a Cegueira - Página: 1 - Review: 2
Livro 1: Ensaio Sobre a Cegueira - Página: 1 - Review: 3
Livro 1: Ensaio Sobre a Cegueira - Página: 1 - Review: 4
Livro 1: Ensaio Sobre a Cegueira - Página: 1 - Review: 5
Livro 1: Ensaio Sobre a Cegueira - Página: 1 - Review: 6
Livro 1: Ensaio Sobre a Cegueira - Página: 1 - Review: 7
Livro 1: Ensaio Sobre a Cegueira - Página: 1 - Review: 8
Livro 1: Ensaio Sobre a Cegueira - Página: 1 - Review: 9
Livro 1: Ensaio Sobre a Cegueira - Página: 1 - Review: 10
Livro 2: O Caçador de Pipas
Livro 2: O Caçador de Pipas - Página: 1
Livro 2: O Caçador de Pipas - Página: 1 - Review: 1
Livro 2: O Caçador de Pipas - Página: 1 - Review: 2
Livro 2: O Caçador de Pipas - Página: 1 - Review: 3
Livro 2: O Caçador de Pipas - Página: 1 - Review: 4
Livro 2: O Caçador de Pipas - Página: 1 - Review: 5
Livro 2: O Caçador de P

In [9]:
df_livros = pd.DataFrame(livros)

In [10]:
df_livros

,autor,titulo,numero_do_livro,generos,data_lancamento,ISBN_13,paginas,nota_media,total_de_avaliacoes,leram,lendo,querem_ler,relendo,abandonos,total_resenhas,url_livro,url_resenha,descricao,reviews
0,José Saramago,Ensaio Sobre a Cegueira,1,"[Ficção, Literatura Estrangeira, Romance]",1995,9788571644953,312,4.6,29343,63355,3055,48402,104,1937,1866,https://www.skoob.com.br/ensaio-sobre-a-ceguei...,https://www.skoob.com.br/livro/resenhas/1/mais...,"\nUma terrível ""treva branca"" vai deixando ceg...","[{'review_id': 'resenha8521829', 'user_id': '2..."
1,Khaled Hosseini,O Caçador de Pipas,2,"[Drama, Literatura Estrangeira, Romance, Ficção]",2005,9788520917671,365,4.3,66313,218486,2272,75558,174,4182,1886,https://www.skoob.com.br/o-caçador-de-pipas-2,https://www.skoob.com.br/livro/resenhas/2/mais...,\nO caçador de pipas é considerado um dos maio...,"[{'review_id': 'resenha493639', 'user_id': '15..."
2,Paulo Coelho,O Alquimista,3,"[Literatura Brasileira, Romance, Autoajuda, Fi...",1989,9788576651857,192,3.8,35340,116366,924,11076,126,1361,1262,https://www.skoob.com.br/o-alquimista-3,https://www.skoob.com.br/livro/resenhas/3/mais...,\nO jovem pastor Santiago tem um sonho que se ...,"[{'review_id': 'resenha7926628', 'user_id': '1..."
3,Richard Bach,Fernão Capelo Gaivota,4,"[Contos, Literatura Estrangeira]",1970,9788501060495,80,3.9,7617,15619,138,1728,37,142,270,https://www.skoob.com.br/fernão-capelo-gaivota-4,https://www.skoob.com.br/livro/resenhas/4/mais...,\nPara as pessoas que inventam suas próprias l...,"[{'review_id': 'resenha12944723', 'user_id': '..."
4,J. J. Benítez,Operação Cavalo de Tróia 1,5,[Literatura Estrangeira],1987,9788572720168,568,4.1,3254,6687,513,2631,36,743,127,https://www.skoob.com.br/operação-cavalo-de-tr...,https://www.skoob.com.br/livro/resenhas/5/mais...,\nRelato de um militar e cientista norte-ameri...,"[{'review_id': 'resenha6396856', 'user_id': '1..."
5,Richard Bach,Ilusões,6,[Romance],1977,9788501011947,156,4.1,1689,3460,51,498,11,49,46,https://www.skoob.com.br/ilusões-6,https://www.skoob.com.br/livro/resenhas/6/mais...,\nO subtítulo deste novo livro de Richard Bach...,"[{'review_id': 'resenha10890172', 'user_id': '..."
6,Markus Zusak,A Menina que Roubava Livros,7,"[Drama, Ficção, Literatura Estrangeira, Romanc...",2007,9788598078373,480,4.5,127222,380643,13992,271459,1003,19180,4817,https://www.skoob.com.br/a-menina-que-roubava-...,https://www.skoob.com.br/livro/resenhas/7/mais...,"\nAo perceber que a pequena Liesel Meminger, u...","[{'review_id': 'resenha44507389', 'user_id': '..."
7,Justin Herald,Atitude,8,None,2004,9788588350489,120,3.6,104,215,12,245,1,7,5,https://www.skoob.com.br/atitude-8,https://www.skoob.com.br/livro/resenhas/8/mais...,\nAos 25 anos de idade e com apenas $50 no bol...,"[{'review_id': 'resenha121097654', 'user_id': ..."
8,Justin Herald,Atitude! 2 - O que você está esperando?,9,None,2005,9788588350885,110,3.7,30,62,5,67,0,4,0,https://www.skoob.com.br/atitude-2-o-que-você-...,https://www.skoob.com.br/livro/resenhas/9/mais...,\nQual o sacrifício que você está disposto a f...,"[{'review_id': None, 'user_id': None, 'livro_p..."
9,Mauro Halfeld,Investimentos,10,"[Economia, Finanças]",2001,9788576761594,165,3.8,543,997,45,510,3,10,20,https://www.skoob.com.br/investimentos-10,https://www.skoob.com.br/livro/resenhas/10/mai...,\nGanhar dinheiro não é uma tarefa fácil. Mais...,"[{'review_id': 'resenha2558295', 'user_id': '8..."
